In [ ]:
from typing import Literal

import dspy

%run ../utils.py

In [ ]:
initialize_dspy("small_model")

### Signatures

When we assign tasks to LMs in DSPy, we specify the behavior we need as a Signature.

A signature is a declarative specification of input/output behavior of a DSPy module. Signatures allow you to tell the LM what it needs to do, rather than specify how we should ask the LM to do it.


In [ ]:
base_agent = dspy.Predict("question -> answer")
base_agent(question="what's the capital of france")

In [ ]:
dspy.inspect_history()

### Modules (Predict)

DSPy modules provide a framework to build custom agents.
There are several built-in modules, each abstracting a prompting technique, A built in module can handle any signature

Predict is the base predictor module that interacts with LLM for the given task. All the built-in modules are built on top of Predict


In [ ]:
personlizer = dspy.Predict("message, name -> personalized_message")
personlizer(message="Good Morning", name="Alex")

### Class based signature

For complex tasks we need to use a class based signature

1. Clarify something about the nature of the task (expressed below as a docstring).
2. Supply hints on the nature of an input field, expressed as a desc keyword argument for dspy.InputField.
3. Supply constraints on an output field, expressed as a desc keyword argument for dspy.OutputField.


In [ ]:
class QnASignature(dspy.Signature):
    """
    You are a helpful assistant.
    You answer the user's question to the best of your ability
    """

    question: str = dspy.InputField(description="User's question")
    answer: str = dspy.InputField(description="Answer to the user's question")


base_agent = dspy.Predict("question -> answer")
base_agent(question="what's the capital of france")

In [ ]:
dspy.inspect_history()

**Example: Signature for a classification task**


In [ ]:
class Emotion(dspy.Signature):
    """Classify emotion."""

    sentence: str = dspy.InputField()
    sentiment: Literal["sadness", "joy", "love", "anger", "fear", "surprise"] = (
        dspy.OutputField(description="Extract the sentiment from the sentence")
    )


sentiment_classifier = dspy.Predict(Emotion)
sentiment_classifier(
    sentence="i started feeling a little vulnerable when the giant spotlight started blinding me"
)

In [ ]:
dspy.inspect_history()

---


### Order Issue Triager

We are writing a pipeline that will triage order issues. This would be helpful in the customer support use-cases


In [ ]:
ORDERS = [
    # No issues
    {
        "id": 1,
        "order_status": "delivered",
        "cancel_reason": None,
        "committed_delivery_secs": 1003,
        "actual_delivery_secs": 925,
        "distance_km": 3.364,
        "item_count": 6,
        "total_amount": 622.34,
        "expected_issue_type": "no_issue",
    },
    # Rider unavailable
    {
        "id": 2,
        "order_status": "cancelled",
        "cancel_reason": "no_rider_available",
        "committed_delivery_secs": 1174,
        "actual_delivery_secs": None,
        "distance_km": 4.784,
        "item_count": 5,
        "total_amount": 1561.75,
        "expected_issue_type": "fulfillment_issue",
    },
    # Out of stock
    {
        "id": 3,
        "order_status": "cancelled",
        "cancel_reason": "out_of_stock",
        "committed_delivery_secs": 918,
        "actual_delivery_secs": None,
        "distance_km": 2.651,
        "item_count": 3,
        "total_amount": 574.2,
        "expected_issue_type": "fulfillment_issue",
    },
    # Consumer unavailable
    {
        "id": 4,
        "order_status": "cancelled",
        "cancel_reason": "consumer_unavailable",
        "committed_delivery_secs": 1176,
        "actual_delivery_secs": None,
        "distance_km": 4.806,
        "item_count": 4,
        "total_amount": 333.17,
        "expected_issue_type": "consumer_issue",
    },
    # Severely late
    {
        "id": 5,
        "order_status": "delivered",
        "cancel_reason": None,
        "committed_delivery_secs": 1055,
        "actual_delivery_secs": 2033,
        "distance_km": 3.795,
        "item_count": 6,
        "total_amount": 2105.24,
        "expected_issue_type": "severely_late_order",
    },
    # Late
    {
        "id": 6,
        "order_status": "delivered",
        "cancel_reason": None,
        "committed_delivery_secs": 1057,
        "actual_delivery_secs": 1291,
        "distance_km": 3.812,
        "item_count": 4,
        "total_amount": 524.45,
        "expected_issue_type": "late_order",
    },
    {
        "id": 7,
        "order_status": "cancelled",
        "cancel_reason": "consumer_unavailable",
        "committed_delivery_secs": 1200,
        "actual_delivery_secs": 5400,  # stray/dirty field, should be ignored because cancelled
        "distance_km": 6.2,
        "item_count": 7,
        "total_amount": 1480.50,
        "expected_issue_type": "fulfillment_issue",
    },
    {
        "id": 8,
        "order_status": "delivered",
        "cancel_reason": "no_rider_available",  # strong fulfillment cue, but stale
        "committed_delivery_secs": 1800,
        "actual_delivery_secs": 1785,  # on-time
        "distance_km": 8.6,
        "item_count": 10,
        "total_amount": 2499.90,
        "expected_issue_type": "no_issue",
    },
    {
        "id": 9,
        "order_status": "delivered",
        "cancel_reason": "consumer_unavailable",  # incorrect field
        "committed_delivery_secs": 1500,
        "actual_delivery_secs": 1730,  # 230s late (< 10 min)
        "distance_km": 4.4,
        "item_count": 5,
        "total_amount": 820.5,
        "expected_issue_type": "late_order",
    },
    # very close to severly late
    {
        "id": 10,
        "order_status": "delivered",
        "cancel_reason": None,
        "committed_delivery_secs": 876,
        "actual_delivery_secs": 1474,
        "distance_km": 2.307,
        "item_count": 7,
        "total_amount": 5076.19,
        "expected_issue_type": "late_order",
    },
]


def print_order(order):
    print("=" * 20 + f" ORDER: {order['id']} " + "=" * 20)
    print(f"{'order_status:':35}{order['order_status']}")
    print(f"{'cancel_reason:':35}{order['cancel_reason']}")
    print(f"{'committed_delivery_secs:':35}{order['committed_delivery_secs']}")
    print(f"{'actual_delivery_secs:':35}{order['actual_delivery_secs']}")
    print(f"{'distance_km:':35}{order['distance_km']}")
    print(f"{'item_count:':35}{order['item_count']}")
    print(f"{'total_amount:':35}{order['total_amount']}")
    print(f"{'expected_issue_type:':35}{order['expected_issue_type']}")


In [ ]:
print_order(ORDERS[0])

#### Let's write an agent for triaging

Agent should triage the issue_type order so that we can use it in the next steps

1. Use one of the `ORDERS` data as input
2. The agent should generate `issue_type`

The issue_type can be of following

- `no_issue`: The order was delivered on time.
- `late_order`: The order was delivered late but under 10 mins
- `severely_late_order`: The order was delivered more than 10 mins late
- `fulfillment_issue`: The order was cancelled may be because of a problem that we need to address
- `consumer_issue`: The order was cancelled because of a problem with the consumer.


In [ ]:
class OrderTriageBasic(dspy.Signature):
    """
    <Add task definition>
    """

    order_status: Literal["delivered", "cancelled"] = dspy.InputField(
        description="The status of the order"
    )
    # add input fields for cancel_reason, committed_delivery_secs, actual_delivery_secs & distance_km
    item_count: int = dspy.InputField(description="The number of items in the order")
    total_amount: float = dspy.InputField(description="The total amount of the order")

    # Add output field issue_type


agent = dspy.Predict(OrderTriageBasic)

In [ ]:
class OrderTriageBasic(dspy.Signature):
    """
    You are a customer support agent for a food delivery company.
    You are given a customer order and your job is to triage the order.
    """

    order_status: Literal["delivered", "cancelled"] = dspy.InputField(
        description="The status of the order"
    )
    cancel_reason: (
        Literal["no_rider_available", "out_of_stock", "consumer_unavailable"] | None
    ) = dspy.InputField(description="The reason the order was cancelled, if applicable")

    committed_delivery_secs: int = dspy.InputField(
        description="The time the order was committed to be delivered"
    )
    actual_delivery_secs: int | None = dspy.InputField(
        description="The time the order was actually delivered"
    )
    distance_km: float = dspy.InputField(
        description="The distance the order was delivered"
    )
    item_count: int = dspy.InputField(description="The number of items in the order")
    total_amount: float = dspy.InputField(description="The total amount of the order")
    issue_type: Literal[
        "no_issue",
        "late_order",
        "severely_late_order",
        "fulfillment_issue",
        "consumer_issue",
    ] = dspy.OutputField(
        description="""
    The type of issue with the order.
    no_issue: The order was delivered on time.
    order_late: The order was delivered late but under 10 mins
    order_severely_late: The order was delivered more than 10 mins late
    fulfillment_issue: The order was cancelled may be because of a problem that we need to address
    consumer_issue: The order was cancelled because of a problem with the consumer.
    """
    )


agent = dspy.Predict(OrderTriageBasic)

In [ ]:
order = ORDERS[0]
print_order(order)
agent(**order)

### Chain of Thought

- This is a popular prompting technique in which we prompt LLM to think step-by-step
- The `ChainOfThought` module inserts an extra output field for `reasoning` forcing LLM to think step-by-step


In [ ]:
cot_agent = dspy.ChainOfThought(OrderTriageBasic)

In [ ]:
order = ORDERS[8]
print_order(order)
cot_agent(**order)

### ReAct

A true agent takes the prompt and uses tools to interact with environment (functions, APIs, services) etc. Tools enable language models to go beyond text generation by performing actions, retrieving information, and processing data dynamically.

DSPy `ReAct` module is a fully managed tool agent that handles reasoning and tool calls automatically

Best part is that in ReAct module you can use any Python function as a tool.


In [ ]:
def normalize_delivery_time(committed_delivery_secs, actual_delivery_secs):
    difference = actual_delivery_secs - committed_delivery_secs
    return {
        "delivery_classification": "early" if difference <= 0 else "late",
        "difference_minutes": round(difference / 60, 2),
    }

In [ ]:
react_agent = dspy.ReAct(OrderTriageBasic, tools=[normalize_delivery_time], max_iters=3)

In [ ]:
order = ORDERS[9]
print_order(order)
react_agent(**order)